## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Porsche Design Tower\r18555 Collins Ave 5401\r...,4/17/2023,57,"$6,500,000.00",3171,"$2,049.83",Jeffrey Miller,One Sotheby's Int'l Realty,Christina Lawrence,"Compass Florida, LLC",Porsche Design Tower 18555 Collins Ave 5401 Su...,Porsche Design Tower,18555 Collins Ave 5401 Sunny Isles Beach,"(25.9482206, -80.1197921)",25.948221,-80.119792
2,Prive\r5500 Island Estates Dr 1505\rAventura,4/19/2023,37,"$6,000,000.00",3984,"$1,506.02",Randi Rapp,"Compass Florida, LLC",Adriana Faerman,"Compass Florida, LLC",Prive 5500 Island Estates Dr 1505 Aventura,Prive,5500 Island Estates Dr 1505 Aventura,"(25.946531, -80.130636)",25.946531,-80.130636
3,Continuum on South Beach\r100 S Pointe Dr 1108...,4/21/2023,32,"$5,700,000.00",1757,"$3,244.17",John Lennon,South Pointe Drive Realty Inc,Selen Arditi,"Compass Florida, LLC",Continuum on South Beach 100 S Pointe Dr 1108 ...,Continuum on South Beach,100 S Pointe Dr 1108 Miami Beach,"(25.7672315, -80.1327611)",25.767232,-80.132761
4,Turnberry Ocean Colony N\r16051 Collins Ave 32...,4/17/2023,189,"$4,000,000.00",3735,"$1,070.95",Jeannie Montes de Oca,BHHS EWM Realty,Bela Chuzhyk,Bela Chuzhyk,Turnberry Ocean Colony N 16051 Collins Ave 320...,Turnberry Ocean Colony N,16051 Collins Ave 3204 Sunny Isles Beach,"(25.926267, -80.12156499999999)",25.926267,-80.121565
5,Grand Bay Tower\r430 Grand Bay Dr 1205\rKey Bi...,4/18/2023,151,"$3,600,000.00",2930,"$1,228.67",Bruno Junqueira,Coldwell Banker Realty,Maria Arguello PA,Envirotek Realty,Grand Bay Tower 430 Grand Bay Dr 1205 Key Bisc...,Grand Bay Tower,430 Grand Bay Dr 1205 Key Biscayne,"(25.693091, -80.157996)",25.693091,-80.157996
6,Icon Condo\r450 Alton Rd 801\rMiami Beach,4/18/2023,60,"$3,100,000.00",2145,"$1,445.22",Jennifer Goldstein,Official,NON-MLS MEMBER,MAR NON MLS MEMBER,Icon Condo 450 Alton Rd 801 Miami Beach,Icon Condo,450 Alton Rd 801 Miami Beach,"(25.7737477, -80.14112109999999)",25.773748,-80.141121
7,Turnberry Ocean Colony\r16047 Collins Ave 501\...,4/19/2023,362,"$2,700,000.00",2772,$974.03,Phillip Tartakovsky,Silmar Realty,Anastasia Mastantuono,Miami Realty Solution Group,Turnberry Ocean Colony 16047 Collins Ave 501 S...,Turnberry Ocean Colony,16047 Collins Ave 501 Sunny Isles Beach,"(25.9252763, -80.1216546)",25.925276,-80.121655
8,Mosaic\r3801 Collins Ave 506\rMiami Beach,4/21/2023,148,"$2,550,000.00",1820,"$1,401.10",Steven Koleno,Beycome of Florida LLC,Taylor Richardson,The Corcoran Group,Mosaic 3801 Collins Ave 506 Miami Beach,Mosaic,3801 Collins Ave 506 Miami Beach,"(25.8113373, -80.12262849999999)",25.811337,-80.122628
9,Bella Mare\r6000 Island Blvd 1902\rAventura,4/18/2023,72,"$2,290,000.00",2840,$806.34,Richard Goihman,Douglas Elliman,Carlos Escoto,Douglas Elliman,Bella Mare 6000 Island Blvd 1902 Aventura,Bella Mare,6000 Island Blvd 1902 Aventura,"(25.9427325, -80.1352515)",25.942733,-80.135251
10,0Peloro\r6620 Indian Creek Dr 710\rMiami Beach,4/17/2023,57,"$2,000,000.00",2417,$827.47,Romina Grinberg,Brown Harris Stevens,Eitan Gontovnik,"Compass Florida, LLC",0Peloro 6620 Indian Creek Dr 710 Miami Beach,,0Peloro 6620 Indian Creek Dr 710 Miami Beach,"(25.8502035, -80.1219914)",25.850203,-80.121991


In [9]:
df.at[10,'building_name']=('Peloro')
df.at[10,'address_only']=('6620 Indian Creek Drive')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 16th - April 22nd')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [18]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Porsche Design Tower , 18555 Collins Ave 5401 Sunny Isles Beach | Price $6,500,000 | $2,050 psf | Listing agent: Jeffrey Miller with One Sotheby's Int'l Realty | Buyer's agent: Christina Lawrence with Compass Florida, LLC | Days on market: 57
Least Expensive
Peloro, 6620 Indian Creek Drive | Price $2,000,000 | $827 psf | Listing agent: Romina Grinberg with Brown Harris Stevens | Buyer's agent: Eitan Gontovnik with Compass Florida, LLC | Days on market: 57
Highest Price Per Square Foot
Continuum on South Beach , 100 S Pointe Dr 1108 Miami Beach | Price $5,700,000 | $3,244 psf | Listing agent: John Lennon with South Pointe Drive Realty Inc | Buyer's agent: Selen Arditi with Compass Florida, LLC | Days on market: 32
Lowest Price Per Square Foot
Bella Mare , 6000 Island Blvd 1902 Aventura | Price $2,290,000 | $806 psf | Listing agent: Richard Goihman with Douglas Elliman | Buyer's agent: Carlos Escoto with Douglas Elliman | Days on market: 72
Most Days on Market
Turnberry Oc

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_04222023
